<a href="https://colab.research.google.com/github/qurrata111/OpenNMT-py/blob/master/docs/source/examples/Translation_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install OpenNMT-py 2.x
# NOTE: By the end of the insatallation, it might ask for restarting the runtime...
# In this case, just click the "RESTART RUNTIME" button.

!pip3 install OpenNMT-py

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)

1.9.0+cu102
10.2


In [9]:
# Optional: List the extracted files

!ls drive/MyDrive/dataset/en_sunda

src-test.txt   src-val.txt   tgt-train.txt
src-train.txt  tgt-test.txt  tgt-val.txt


In [10]:
# Optional: Print the first 3 lines of the source file

!head -n 3 drive/MyDrive/dataset/en_sunda/src-train.txt

Opening: 2007 year in review 
2007 was another significant year for the Australia Indonesia Partnership. 
Reconstruction after the 2004 Boxing Day tsunami neared completion, helping to create a more positive future for communities in Aceh and Nias. 


In [11]:
# Optional: Check the number of lines in the source file

!echo "Number of lines:" && wc -l drive/MyDrive/dataset/en_sunda/src-train.txt

Number of lines:
53933 drive/MyDrive/dataset/en_sunda/src-train.txt


In [12]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano

# TO DO: increasing train steps & validation step

config = '''# toy_en_de.yaml

## Where the samples will be written
save_data: drive/MyDrive/dataset/en_sunda/run/example

## Where the vocab(s) will be written
src_vocab: drive/MyDrive/dataset/en_sunda/run/example.vocab.src
tgt_vocab: drive/MyDrive/dataset/en_sunda/run/example.vocab.tgt

## Where the model will be saved
save_model: model/model

# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: drive/MyDrive/dataset/en_sunda/src-train.txt
        path_tgt: drive/MyDrive/dataset/en_sunda/tgt-train.txt
    valid:
        path_src: drive/MyDrive/dataset/en_sunda/src-val.txt
        path_tgt: drive/MyDrive/dataset/en_sunda/tgt-val.txt

world_size: 1
gpu_ranks: [0]

# Remove or modify these lines for bigger files
train_steps: 3000
valid_steps: 300

early_stopping: 5
'''

with open("toy_en_de.yaml", "w+") as config_yaml:
  config_yaml.write(config)

!cat toy_en_de.yaml

# toy_en_de.yaml

## Where the samples will be written
save_data: drive/MyDrive/dataset/en_sunda/run/example

## Where the vocab(s) will be written
src_vocab: drive/MyDrive/dataset/en_sunda/run/example.vocab.src
tgt_vocab: drive/MyDrive/dataset/en_sunda/run/example.vocab.tgt

## Where the model will be saved
save_model: model/model

# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: drive/MyDrive/dataset/en_sunda/src-train.txt
        path_tgt: drive/MyDrive/dataset/en_sunda/tgt-train.txt
    valid:
        path_src: drive/MyDrive/dataset/en_sunda/src-val.txt
        path_tgt: drive/MyDrive/dataset/en_sunda/tgt-val.txt

world_size: 1
gpu_ranks: [0]

# Remove or modify these lines for bigger files
train_steps: 3000
valid_steps: 300

early_stopping: 5


In [13]:
# Build Vocabulary

!onmt_build_vocab -config toy_en_de.yaml -n_sample -1

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-09-22 16:08:16,508 INFO] Counter vocab from -1 samples.
[2021-09-22 16:08:16,508 INFO] n_sample=-1: Build vocab on full datasets.
[2021-09-22 16:08:16,517 INFO] corpus_1's transforms: TransformPipe()
[2021-09-22 16:08:18,237 INFO] Counters src:64526
[2021-09-22 16:08:18,237 INFO] Counters tgt:62895


In [14]:
# Check if GPU is active
# If not, go to "Runtime" menu > "Change runtime type" > "GPU"

!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-daee32af-2353-de9a-67e8-a73e1c2945d8)


In [15]:
# Make sure the GPU is visable to PyTorch

import torch

gpu_id = torch.cuda.current_device()
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(gpu_id))

True
Tesla K80


In [16]:
# Train the NMT model

!onmt_train -config toy_en_de.yaml

[2021-09-22 16:09:05,735 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-09-22 16:09:05,736 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-09-22 16:09:05,736 INFO] Missing transforms field for valid data, set to default: [].
[2021-09-22 16:09:05,737 INFO] Parsed 2 corpora from -data.
[2021-09-22 16:09:05,738 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-09-22 16:09:05,738 INFO] Loading vocab from text file...
[2021-09-22 16:09:05,738 INFO] Loading src vocabulary from drive/MyDrive/dataset/en_sunda/run/example.vocab.src
[2021-09-22 16:09:05,898 INFO] Loaded src vocab has 64526 tokens.
[2021-09-22 16:09:05,933 INFO] Loading tgt vocabulary from drive/MyDrive/dataset/en_sunda/run/example.vocab.tgt
[2021-09-22 16:09:06,084 INFO] Loaded tgt vocab has 62895 tokens.
[2021-09-22 16:09:06,120 INFO] Building fields with vocab in counters...
[2021-09-22 16:09:06,252 INFO]  * tgt vocab size: 50004.
[20

In [17]:
# Optional: Make a smaller file with 50 sentences instead of 2737
!head -20 drive/MyDrive/dataset/en_sunda/src-test.txt > drive/MyDrive/dataset/en_sunda/src-test-20.txt

!wc -l drive/MyDrive/dataset/en_sunda/src-test.txt
!wc -l drive/MyDrive/dataset/en_sunda/src-test-20.txt

6964 drive/MyDrive/dataset/en_sunda/src-test.txt
20 drive/MyDrive/dataset/en_sunda/src-test-20.txt


In [18]:
# Translate

!onmt_translate -model model/model_step_3000.pt -src drive/MyDrive/dataset/en_sunda/src-test.txt -output drive/MyDrive/dataset/en_sunda/pred_1000.txt -gpu 0 -verbose

Streaming output truncated to the last 5000 lines.
SENT 5966: ['In', 'order', 'to', 'become', 'a', 'certification', 'implementer', 'LPTKs', 'fulfilling', 'the', 'requirements', 'were', 'required', 'to', 'put', 'forward', 'a', 'proposal', 'to', 'MoNE', 'for', 'assessment']
PRED 5966: Dina raraga jadi hiji hiji dana anu anu diperlukeun pikeun nyandak hiji hiji perjangjian pikeun sertifikasi pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun layanan pikeun l